# Forecasting Methods in TimeSmith

This notebook showcases the diverse forecasting methods available in TimeSmith, from simple moving averages to advanced methods like ARIMA, Prophet, and Bayesian forecasting.

## What You'll Learn

- Multiple forecasting algorithms
- Model comparison and selection
- Uncertainty quantification
- Performance evaluation


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from timesmith import (
    ForecastTask,
    SimpleMovingAverageForecaster,
    ExponentialMovingAverageForecaster,
    ExponentialSmoothingForecaster,
    compare_models,
    ModelComparison,
)
from timesmith.eval import backtest_forecaster, summarize_backtest

np.random.seed(42)
plt.style.use('seaborn-v0_8-darkgrid')


## Create Realistic Time Series Data

Let's create a time series with trend, seasonality, and noise.


In [ ]:
# Create monthly data with trend and seasonality
dates = pd.date_range("2018-01-01", periods=60, freq="M")

# Trend
trend = np.linspace(100, 200, len(dates))

# Strong seasonal pattern (annual)
seasonal = 30 * np.sin(2 * np.pi * np.arange(len(dates)) / 12)

# Noise
noise = np.random.normal(0, 10, len(dates))

# Combine
y = pd.Series(trend + seasonal + noise, index=dates, name="sales")

# Plot
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(y.index, y.values, linewidth=2, label="Sales", color="steelblue")
ax.set_title("Monthly Sales Data (with Trend and Seasonality)", fontsize=16, fontweight="bold")
ax.set_xlabel("Date", fontsize=12)
ax.set_ylabel("Sales", fontsize=12)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"Data Statistics:")
print(f"  Mean: {y.mean():.2f}")
print(f"  Std: {y.std():.2f}")
print(f"  Min: {y.min():.2f}")
print(f"  Max: {y.max():.2f}")


## Compare Multiple Forecasters

TimeSmith makes it easy to compare different forecasting methods.


In [ ]:
# Create forecasters
forecasters = {
    "Simple MA (7)": SimpleMovingAverageForecaster(window=7),
    "Simple MA (12)": SimpleMovingAverageForecaster(window=12),
    "Exponential MA": ExponentialMovingAverageForecaster(alpha=0.3),
    "Exponential Smoothing": ExponentialSmoothingForecaster(),
}

# Create task
task = ForecastTask(y=y, fh=12, frequency="M")

# Compare models
comparison = compare_models(
    forecasters=forecasters,
    task=task,
    metrics=["mae", "rmse", "mape"]
)

print("Model Comparison Results:")
print("=" * 70)
print(comparison.to_string())


## Visualize Forecasts from Different Models

Let's see how different models forecast the future.


In [ ]:
# Fit all models and generate forecasts
forecasts = {}
for name, forecaster in forecasters.items():
    forecaster.fit(y)
    forecast = forecaster.predict(fh=12)
    forecasts[name] = forecast

# Plot
fig, ax = plt.subplots(figsize=(16, 8))

# Plot historical data
ax.plot(y.index[-24:], y.values[-24:], 'o-', label="Historical", 
        linewidth=2, markersize=6, color="black", alpha=0.7)

# Plot forecasts
colors = plt.cm.tab10(np.linspace(0, 1, len(forecasts)))
for (name, forecast), color in zip(forecasts.items(), colors):
    ax.plot(forecast.y_pred.index, forecast.y_pred.values, 
           '--', label=f"{name} Forecast", linewidth=2, alpha=0.8, color=color)

ax.axvline(x=y.index[-1], color="red", linestyle=":", linewidth=2, 
          alpha=0.7, label="Cutoff")
ax.set_title("Forecast Comparison: Multiple Models", fontsize=16, fontweight="bold")
ax.set_xlabel("Date", fontsize=12)
ax.set_ylabel("Sales", fontsize=12)
ax.legend(loc="best", fontsize=10)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## Backtest and Evaluate Models

Use time series cross-validation to properly evaluate forecasters.


In [ ]:
# Run backtests for each model
results = {}
for name, forecaster in forecasters.items():
    result = backtest_forecaster(forecaster, task)
    summary = summarize_backtest(result)
    results[name] = summary["aggregate_metrics"]
    print(f"\n{name}:")
    print(f"  MAE: {summary['aggregate_metrics']['mean_mae']:.2f}")
    print(f"  RMSE: {summary['aggregate_metrics']['mean_rmse']:.2f}")
    print(f"  MAPE: {summary['aggregate_metrics']['mean_mape']:.2f}%")

# Create comparison DataFrame
comparison_df = pd.DataFrame(results).T
print("\n" + "=" * 70)
print("Summary Comparison:")
print(comparison_df.round(2))


## Advanced: Exponential Smoothing with Prediction Intervals

Some forecasters support uncertainty quantification.


In [ ]:
# Try to get prediction intervals (if supported)
try:
    es_forecaster = ExponentialSmoothingForecaster()
    es_forecaster.fit(y)
    
    # Check if predict_interval is supported
    if hasattr(es_forecaster, 'predict_interval'):
        forecast_with_intervals = es_forecaster.predict_interval(fh=12, coverage=0.95)
        
        # Plot with intervals
        fig, ax = plt.subplots(figsize=(14, 7))
        ax.plot(y.index[-24:], y.values[-24:], 'o-', label="Historical", 
               linewidth=2, markersize=6, color="black")
        
        if hasattr(forecast_with_intervals, 'y_int') and forecast_with_intervals.y_int is not None:
            lower = forecast_with_intervals.y_int.iloc[:, 0]
            upper = forecast_with_intervals.y_int.iloc[:, 1]
            ax.fill_between(forecast_with_intervals.y_pred.index, 
                           lower, upper, alpha=0.3, color="steelblue", 
                           label="95% Prediction Interval")
        
        ax.plot(forecast_with_intervals.y_pred.index, forecast_with_intervals.y_pred.values,
               '--', label="Forecast", linewidth=2, color="steelblue")
        ax.axvline(x=y.index[-1], color="red", linestyle=":", linewidth=2)
        ax.set_title("Forecast with Prediction Intervals", fontsize=16, fontweight="bold")
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    else:
        print("Prediction intervals not available for this forecaster")
except Exception as e:
    print(f"Prediction intervals not available: {e}")


## Summary

You've learned:
- How to use multiple forecasting methods
- How to compare models using TimeSmith's comparison tools
- How to visualize forecasts from different models
- How to evaluate models with backtesting
- How to work with prediction intervals (when available)

**Next Steps:**
- Try optional forecasters (ARIMA, Prophet, LSTM) if installed
- Explore Monte Carlo forecasting for uncertainty
- Learn about ensemble methods
